In [1]:
from sqlalchemy import create_engine
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime

In [2]:
# Configure settings for RDS
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}

In [3]:
#postgres://[user]:[password]@[location]:[port]/[database]
create_engine_str = 'postgresql://' + config["user"] + ":" + config["password"] + "@" + config["location"] + ":" + config["port"] + "/" + config["db"]

In [4]:
engine = create_engine(create_engine_str)

In [5]:
print (engine.table_names())

['education', 'committee_summary_2020', 'fec_donor_az', 'health_metrics', 'birth_death_rate', 'postal_codes', 'fec_donor_mi', 'fec_donor_wi', 'fec_committee', 'fec_donor_pa', 'pres_votes_6t', 'unemployment', 'fec_donor_nc', 'fec_donor_fl']


In [6]:
def get_votes_intervals(votes_df, state_po):
    votes_states_df = votes_df[votes_df['state_po']==state_po]
    starting_yr = 2000
    ending_yr = 2020
    interval = 4
    i = starting_yr
    
    four_yr_dfs = []
    while (i <= ending_yr):
        votes_states_interval_df = votes_states_df[votes_states_df['year']==i]    
        four_yr_dfs.append(votes_states_interval_df)
        i += interval
    
    return four_yr_dfs

In [7]:
def vote_distribution(county, election_df):
    major_parties = ["democrat", "republican"]
    
    print(election_df.columns)
    
    county_df = election_df[election_df['county']==county]
    #TODO get the party flag.
    county_blue_df = county_df[county_df['party']==major_parties[0]]
    county_red_df = county_df[county_df['party']==major_parties[1]]  
    
    #Other = not democratic AND not republican  
    other_votes = 0
    blue_votes = pd.to_numeric(county_blue_df["candidatevotes"].sum(), errors='coerce')
    red_votes = pd.to_numeric(county_red_df["candidatevotes"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in major_parties:
            party_df = county_df[county_df['party']==party]
            other_votes += pd.to_numeric(party_df["candidatevotes"].sum(), errors='coerce')
    
    #Total votes it the sum of blue + red + other
    total_votes = blue_votes + red_votes + other_votes
    
    #Get the respective percentages
    percent_blue = (blue_votes / total_votes)
    percent_red = (red_votes / total_votes)
    percent_other = (other_votes / total_votes)
            
    percent_dict = {
        "blue_votes": blue_votes,
        "red_votes": red_votes,
        "other_votes": other_votes,
        "total_votes": total_votes,
        "percent_blue": percent_blue,
        "percent_red": percent_red,
        "percent_other": percent_other
    }
    return percent_dict

In [8]:
def donor_distribution(county, election_df):
    #Total sum of donations per party per county
    major_parties = ["democrat", "republican"]
    
    print(election_df.columns)
    
    county_df = election_df[election_df['county']==county]
    #TODO get the party flag.
    county_blue_df = county_df[county_df['party']==major_parties[0]]
    county_red_df = county_df[county_df['party']==major_parties[1]]  
    
    #Other = not democratic AND not republican  
    others_amt = 0
    blue_amt = pd.to_numeric(county_blue_df["transaction_amt"].sum(), errors='coerce')
    red_amt = pd.to_numeric(county_red_df["transaction_amt"].sum(), errors='coerce')
    
    unique_parties = county_df['party'].unique()
    for party in unique_parties:
        #Get a sum of all non major parties for other category
        if party not in major_parties:
            party_df = county_df[county_df['party']==party]
            others_amt += pd.to_numeric(party_df["transaction_amt"].sum(), errors='coerce')
    
    #Total transaction amount it the sum of blue + red + other
    total_amt = blue_amt + red_amt + others_amt
    
    #Get the respective percentages
    percent_blue = (blue_amt / total_amt)
    percent_red = (red_amt / total_amt)
    percent_other = (others_amt / total_amt)
            
    percent_dict = {
        "blue_amt": blue_amt,
        "red_amt": red_amt,
        "other_amt": others_amt,
        "total_amt": total_amt,
        "percent_blue": percent_blue,
        "percent_red": percent_red,
        "percent_other": percent_other
    }
    return percent_dict

In [9]:
def county_vote_distribution(four_yr_dfs):
    #Organize by county
    county_dicts = []
    for i in range(len(four_yr_dfs)):
        county_dict = {}
        election_df = four_yr_dfs[i]
        unique_counties = election_df["county"].unique()
        for county in unique_counties:
            percent_dict = vote_distribution(county, election_df)
            county_dict[county] = percent_dict
        county_dicts.append(county_dict)
    return county_dicts

In [10]:
"""
ACE	Ace Party	
AKI	Alaskan Independence Party	
AIC	American Independent Conservative	
AIP	American Independent Party	
AMP	American Party	
APF	American People's Freedom Party	
AE	Americans Elect	
CIT	Citizens' Party	
CMD	Commandments Party	
CMP	Commonwealth Party of the U.S.	
COM	Communist Party	
CNC	Concerned Citizens Party Of Connecticut	
CRV	Conservative Party	
CON	Constitution Party	
CST	Constitutional	
COU	Country	
DCG	D.C. Statehood Green Party	
DNL	Democratic -Nonpartisan League	
DEM	Democratic Party	
D/C	Democratic/Conservative	
DFL	Democratic-Farmer-Labor	
DGR	Desert Green Party	
FED	Federalist	
FLP	Freedom Labor Party	
FRE	Freedom Party	
GWP	George Wallace Party	
GRT	Grassroots	
GRE	Green Party	
GR	Green-Rainbow	
HRP	Human Rights Party	
IDP	Independence Party	
IND	Independent	
IAP	Independent American Party	
ICD	Independent Conservative Democratic	
IGR	Independent Green	
IP	Independent Party	
IDE	Independent Party of Delaware	
IGD	Industrial Government Party	
JCN	Jewish/Christian National	
JUS	Justice Party	
LRU	La Raza Unida	Also see RUP
LBR	Labor Party	Also see LAB
LFT	Less Federal Taxes	
LBL	Liberal Party	
LIB	Libertarian Party	
LBU	Liberty Union Party	
MTP	Mountain Party	
NDP	National Democratic Party	
NLP	Natural Law Party	
NA	New Alliance	
NJC	New Jersey Conservative Party	
NPP	New Progressive Party	
NPA	No Party Affiliation	
NOP	No Party Preference	Commonly used in CA & WA
NNE	None	
N	Nonpartisan	
NON	Non-Party	
OE	One Earth Party	
OTH	Other	
PG	Pacific Green	
PSL	Party for Socialism and Liberation	
PAF	Peace And Freedom	Also see PFP
PFP	Peace And Freedom Party	Also see PAF
PFD	Peace Freedom Party	
POP	People Over Politics	
PPY	People's Party	
PCH	Personal Choice Party	
PPD	Popular Democratic Party	
PRO	Progressive Party	
NAP	Prohibition Party	
PRI	Puerto Rican Independence Party	
RUP	Raza Unida Party	Also see LRU
REF	Reform Party	
REP	Republican Party	
RES	Resource Party	
RTL	Right To Life	
SEP	Socialist Equality Party	
SLP	Socialist Labor Party	
SUS	Socialist Party	
SOC	Socialist Party U.S.A.	
SWP	Socialist Workers Party	
TX	Taxpayers	
TWR	Taxpayers Without Representation	
TEA	Tea Party	
THD	Theo-Democratic	
LAB	U.S. Labor Party	Also see LBR
USP	U.S. People's Party	
UST	U.S. Taxpayers Party	
UN	Unaffiliated	
UC	United Citizen	
UNI	United Party	
UNK	Unknown	
VET	Veterans Party	
WTP	We the People	
W	Write-In
"""

"\nACE\tAce Party\t\nAKI\tAlaskan Independence Party\t\nAIC\tAmerican Independent Conservative\t\nAIP\tAmerican Independent Party\t\nAMP\tAmerican Party\t\nAPF\tAmerican People's Freedom Party\t\nAE\tAmericans Elect\t\nCIT\tCitizens' Party\t\nCMD\tCommandments Party\t\nCMP\tCommonwealth Party of the U.S.\t\nCOM\tCommunist Party\t\nCNC\tConcerned Citizens Party Of Connecticut\t\nCRV\tConservative Party\t\nCON\tConstitution Party\t\nCST\tConstitutional\t\nCOU\tCountry\t\nDCG\tD.C. Statehood Green Party\t\nDNL\tDemocratic -Nonpartisan League\t\nDEM\tDemocratic Party\t\nD/C\tDemocratic/Conservative\t\nDFL\tDemocratic-Farmer-Labor\t\nDGR\tDesert Green Party\t\nFED\tFederalist\t\nFLP\tFreedom Labor Party\t\nFRE\tFreedom Party\t\nGWP\tGeorge Wallace Party\t\nGRT\tGrassroots\t\nGRE\tGreen Party\t\nGR\tGreen-Rainbow\t\nHRP\tHuman Rights Party\t\nIDP\tIndependence Party\t\nIND\tIndependent\t\nIAP\tIndependent American Party\t\nICD\tIndependent Conservative Democratic\t\nIGR\tIndependent Green\t\

In [11]:
def map_zip_county(unique_zips, state_zips):
    county_dict = {}
    unique_counties = {}
    for zipcode in unique_zips:
        county_zip = state_zips[state_zips["zip"] == zipcode]
        county_name = county_zip["county"].to_string()
        county_dict[zipcode] = county_name
        if county_name not in unique_counties:
            unique_counties[county_name] = True
            
    return (county_dict, unique_counties.keys())

In [12]:
def map_cmtid_party(election_df, committee_df):    
    merged_df = committee_df.merge(election_df, left_on='cmte_id', right_on='cmt_id')
    
    #committee_df["cmte_pty_affiliation"]
    #committee_df["cmte_id"]
    #election_df["cmt_id"]
    #election_df["cmte_pty_affiliation"] = election_df["cmt_id"].map()
    
    party_repub = "republican"
    party_democrat = "democrat"
        
    #Map the affiliation code to the party affiliation
    cmte_party_map = {
        "REP": party_repub,
        "TEA": party_repub,
        "DNL": party_democrat,
        "DNL": party_democrat,
        "DEM": party_democrat,
        "D/C": party_democrat,
        "DFL": party_democrat,
        "THD": party_democrat,
        "PPD": party_democrat
    }
    
    merged_df["party"] = merged_df["cmte_pty_affiliation"].map(cmte_party_map)
    
    return merged_df

In [13]:
def donation_county_cycle_distribution(four_yr_dfs, state_zips, committee_df):
    #Organize by county
    county_dicts = []
    for i in range(len(four_yr_dfs)):
        county_dict = {}
        election_df = four_yr_dfs[i]
        election_df.dropna(subset=["zip"], inplace=True)
            
        unique_zips = election_df["zip"].unique()
        
        (zip_county_map, unique_counties) = map_zip_county(unique_zips, state_zips)
        print(zip_county_map)
        print("-------")
        print(unique_counties)
        print("-------")
        print(len(zip_county_map.values()))
        print(len(unique_counties))
        
        election_df["county"] = election_df["zip"].map(zip_county_map)
        
        merged_df = map_cmtid_party(election_df, committee_df)
        
        for county in unique_counties:
            percent_dict = donor_distribution(county, merged_df)
            county_dict[county] = percent_dict
        county_dicts.append(county_dict)
    return county_dicts

In [14]:
def str_dt(donor_date_str):
    #01/01/1996 - 12/31/1999
    donor_date = datetime.strptime(donor_date_str, '%Y-%m-%d')
    return donor_date

In [15]:
def get_year_from_date_str(donor_date_str):
    donor_date = str_dt(donor_date_str)
    donor_year = donor_date.year
    return donor_year

In [16]:
def get_donors_intervals(donor_df, state):
    donors_states_df = donor_df[donor_df['state']==state.lower()]
    
    starting_yr = 2000
    i = starting_yr
    interval = 4
    prev_year = starting_yr - interval
    ending_yr = 2020
    
    four_yr_dfs = []
        
    while (i <= ending_yr):
        votes_states_interval_df = donors_states_df[(donors_states_df['transaction_dt']>datetime.date(prev_year,1,1)) & (donors_states_df['transaction_dt']<datetime.date(i,3,1))]  
        print(votes_states_interval_df.head())
        
        four_yr_dfs.append(votes_states_interval_df)
        i += interval
        prev_year += interval
        
    return four_yr_dfs

In [17]:
health_df = pd.read_sql_query('select * from "health_metrics"',con=engine)
health_df.head()

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
0,1000,Alabama,None,81791.0,9942.794666,9840.535949,10045.053384,NaN,NaN,NaN,...,4.443264,3197324,65.413428,48517,1.061048,1.006759,1.115337,51.633032,1957932.0,40.963183
1,42067,Pennsylvania,Juniata,301.0,6943.173764,5646.653331,8239.694197,1.0,NaN,NaN,...,4.003400,23202,93.920013,382,1.649253,0.998613,2.299893,49.866418,20264.0,82.253613
2,42069,Pennsylvania,Lackawanna,3282.0,8878.231356,8401.648792,9354.813920,4.0,NaN,NaN,...,8.102736,177750,84.324432,4543,2.268653,1.929494,2.607811,51.454745,34965.0,16.305488
3,46021,South Dakota,Campbell,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,2.033406,1316,95.570080,0,0.000000,0.000000,3.124080,48.002905,1466.0,100.000000
4,42071,Pennsylvania,Lancaster,5459.0,6128.340367,5882.890633,6373.790101,1.0,NaN,NaN,...,10.810458,443304,81.556120,11626,2.310739,2.099442,2.522036,51.004954,110419.0,21.257111


In [18]:
committee_df = pd.read_sql_query('select * from "fec_committee"',con=engine)
committee_df.head()

,cmte_id,cmte_nm,cmte_tp,cmte_city,cmte_st,cmte_zip,cmte_dsgn,cmte_pty_affiliation,org_tp,connected_org_nm,cand_id
0,C00000059,HALLMARK CARDS PAC,Q,KANSAS CITY,MO,64108,U,UNK,C,None,None
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,Q,WASHINGTON,DC,200017400,B,None,M,DELAWARE MEDICAL PAC,None
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,N,OKLAHOMA CITY,OK,73107,U,None,L,None,None
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,Q,TOPEKA,KS,666121627,U,UNK,M,KANSAS MEDICAL SOCIETY,None
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,Q,INDIANAPOLIS,IN,46202,U,None,M,None,None


In [19]:
votes_df = pd.read_sql_query('select * from "pres_votes_6t"',con=engine)
votes_df.head(100)

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache,4001,President,Al Gore,democrat,13025,19456,20191203
1,2000,Arizona,AZ,Apache,4001,President,George W. Bush,republican,5947,19456,20191203
2,2000,Arizona,AZ,Apache,4001,President,Ralph Nader,green,245,19456,20191203
3,2000,Arizona,AZ,Apache,4001,President,Other,NA,239,19456,20191203
4,2000,Arizona,AZ,Cochise,4003,President,Al Gore,democrat,13360,33241,20191203
...,...,...,...,...,...,...,...,...,...,...,...
95,2000,Florida,FL,Citrus,12017,President,Other,NA,535,57206,20191203
96,2000,Florida,FL,Clay,12019,President,Al Gore,democrat,14632,57353,20191203
97,2000,Florida,FL,Clay,12019,President,George W. Bush,republican,41736,57353,20191203
98,2000,Florida,FL,Clay,12019,President,Ralph Nader,green,562,57353,20191203


In [20]:
zips_df = pd.read_sql_query('select * from "postal_codes"',con=engine)
zips_df.head()

,zip,county,state,stcountyfp,classfp
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [21]:
#List of swing states to run the analysis on
supported_states = ["AZ", "MI", "FL", "NC", "PA", "WI"]

In [22]:
#Loop through each state
for state in supported_states:
    #Get the votes related to that state
    votes_intervals_df = get_votes_intervals(votes_df, state)
    #Get the distribution of Red, Blue, and Other votes in a list of dict per election yr e.g. 2000 + 4n
    counties_votes_dicts = county_vote_distribution(votes_intervals_df)
    
    #Run queries to get all donation records from the states into dfs
    donor_table_name = '"fec_donor_{}"'.format(state.lower())    
    donor_select_sql = 'select * from {}'.format(donor_table_name)
    donor_df = pd.read_sql_query(donor_select_sql,con=engine)
    
    donors_intervals_df = get_donors_intervals(donor_df, state)
    state_zips = zips_df[zips_df["state"] == state]
    
    donor_dict = donation_county_cycle_distribution(donors_intervals_df, state_zips, committee_df)

Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 

      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 

/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in long_scalars
/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars
/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in long_scalars


{}
-------
dict_keys([])
-------
0
0
{}
-------
dict_keys([])
-------
0
0
{}
-------
dict_keys([])
-------
0
0
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', '

      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 

...

      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 

      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 

      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 'state_po', 'county', 'FIPS', 'office', 'candidate',
       'party', 'candidatevotes', 'totalvotes', 'version'],
      dtype='object')
Index(['year', 'state', 

      dtype='object')
Index(['cmte_id', 'cmte_nm', 'cmte_tp', 'cmte_city', 'cmte_st', 'cmte_zip',
       'cmte_dsgn', 'cmte_pty_affiliation', 'org_tp', 'connected_org_nm',
       'cand_id', 'id', 'cmt_id', 'amndt_ind', 'rpt_tp', 'transaction_pgi',
       'image_num', 'transaction_tp', 'entity_tp', 'last_name', 'first_name',
       'city', 'state', 'zip', 'employer', 'occupation', 'transaction_dt',
       'transaction_amt', 'other_id', 'tran_id', 'file_num', 'memo_cd',
       'memo_text', 'sub id', 'county', 'party'],
      dtype='object')
Index(['cmte_id', 'cmte_nm', 'cmte_tp', 'cmte_city', 'cmte_st', 'cmte_zip',
       'cmte_dsgn', 'cmte_pty_affiliation', 'org_tp', 'connected_org_nm',
       'cand_id', 'id', 'cmt_id', 'amndt_ind', 'rpt_tp', 'transaction_pgi',
       'image_num', 'transaction_tp', 'entity_tp', 'last_name', 'first_name',
       'city', 'state', 'zip', 'employer', 'occupation', 'transaction_dt',
       'transaction_amt', 'other_id', 'tran_id', 'file_num', 'memo_cd',
    

/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in long_scalars
/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars
/Users/C454479/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in long_scalars


In [23]:
#Define the Linear Regression Structured MAchine Learning

In [24]:
model = LinearRegression()

In [25]:
#Define X, y

In [26]:
model.fit(X, y)

NameError: name 'X' is not defined

In [ ]:
y_pred = model.predict(X)
print(y_pred.shape)

In [ ]:
plt.scatter(X, y)
plt.plot(X, y_pred, color='red')
plt.show()

In [ ]:
print(model.coef_)
print(model.intercept_)

In [ ]:
#Neural Networking Code
# Generate our categorical variable list
votes_mi_cat = votes_mi_df.dtypes[votes_mi_df.dtypes == "object"].index.tolist()

In [ ]:
# Check the number of unique values in each column
votes_mi_df[votes_mi_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(votes_mi_df[votes_mi_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(votes_mi_cat)
encode_df.head()

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
#TODO would we be doing a classification to determine what party?
y = votes_mi_df["party"].values
X = votes_mi_df.drop(["party"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
merged_df = pd.merge(df1, df2, 
                     left_on = 'names', 
                     right_on = 'name', 
                     how='left')